# Light field voxels

This notebook investigates properties of light field voxels and the voxel-matrix representation.  We do this by raytracing simple 2D scenes.

## Requirements

This notebook only requires the normal `pylab` distribution -- `matplotlib` and `numpy`, probably.

## Methods

We imaged a 2D scene with several occluding circles from all angles, using Dirac $\delta$ basis functions for the angles.  We used one of two representations for the light field:

1. Two-plane parameterization, used by `Voxel.compute()`

2. Spatial-angular (parallel-beam) parameterization, used by `Voxel.compute_parallel()`

We represented the light field as a matrix: each row was a different image captured in either the two plane (pinhole) or parallel-beam parameterization.

We looked at the singular value decomposition of the light field in each case, and at the SVD of several permuted versions of the light field.

## Results

We saw better concentration of the light field spectra using the parallel-beam setup, especially when using an adaptive shift for every column.  My guess is this is because the parallel-beam parameterization is depth-invariant -- each image is essentially focussed at infinity.

The disadvantage of the parallel-beam parameterization is its relative inefficiency.

In [ ]:
%pylab inline

In [ ]:
class Voxel(object):
    def __init__(self, x0, x1, y0, y1, num_disc):
        self.x0 = x0
        self.y0 = y0
        self.x1 = x1
        self.y1 = y1
        self.num_disc = num_disc
        
    @property
    def north_points(self):
        x_points = linspace(self.x0, self.x1, self.num_disc+2).tolist()[1:-1]
        y_points = [self.y1,] * self.num_disc
        return (x_points, y_points)
    
    @property
    def south_points(self):
        x_points = linspace(self.x1, self.x0, self.num_disc+2).tolist()[1:-1]
        y_points = [self.y0,] * self.num_disc
        return (x_points, y_points)
    
    @property
    def east_points(self):
        x_points = [self.x1,] * self.num_disc
        y_points = linspace(self.y1, self.y0, self.num_disc+2).tolist()[1:-1]
        return (x_points, y_points)
    
    @property
    def west_points(self):
        x_points = [self.x0,] * self.num_disc
        y_points = linspace(self.y0, self.y1, self.num_disc+2).tolist()[1:-1]
        return (x_points, y_points)
    
    @property
    def points(self):
        x_points = []
        y_points = []
        for (xp, yp) in (self.north_points, self.east_points, self.south_points, self.west_points):
            x_points += xp
            y_points += yp
        return (x_points, y_points)
        
    def draw(self, ax):
        # draw border
        ax.plot((self.x0, self.x1, self.x1, self.x0, self.x0),
                (self.y0, self.y0, self.y1, self.y1, self.y0),
                color='black',
                linewidth=2)
        
        # draw spatial interior
        xm = (self.x0 + self.x1)/2.0
        ym = (self.y0 + self.y1)/2.0
        ax.plot((xm, self.x1, xm, self.x0, xm),
                (self.y0, ym, self.y1, ym, self.y0),
                color='blue',
                linestyle='--')
        
        # draw points
        x_points, y_points = self.points
        ax.scatter(x_points, y_points, color='black')
        
    def compute(self, objects, transparent=False):
        import itertools
        import collections
        
        x_points, y_points = self.points
        num_points = len(x_points)
        tr = zeros((num_points, num_points))
        for (j, (xj, yj)) in enumerate(zip(x_points, y_points)):
            column_points = collections.deque(zip(x_points, y_points))
            #column_points.rotate(-j)
            for (i, (xi, yi)) in enumerate(column_points):
                dist = inf
                value = 0
                for obj in objects:
                    di, vi = obj.cast(xj, yj, xi, yi, transparent)
                    if di < dist or transparent:
                        if not transparent:
                            value = vi
                        else:
                            value += vi
                        dist = di
                tr[i,j] = value
        return tr
    
    def compute_parallel(self, objects, transparent=False):
        import itertools
        import collections
        
        x_points, y_points = self.points
        num_points = len(x_points)
        tr = zeros((num_points, num_points))
        for (j, (xj, yj)) in enumerate(zip(x_points, y_points)):
            column_points = collections.deque(zip(x_points, y_points))
            column_points.rotate(-j)
            for (i, (xi, yi)) in enumerate(column_points):
                dist = inf
                value = 0
                for obj in objects:
                    di, vi = obj.cast(xj, yj, xj + 20*xi, xj + 20*yi, transparent)
                    if di < dist or transparent:
                        if not transparent:
                            value = vi
                        else:
                            value += vi
                        dist = di
                tr[i,j] = value
        return tr
    
    def permute(self, L):
        # permute north block
        M = 1*L
        M[:, 0:self.num_disc]
    
    def unpermute(self, M):
        pass

In [ ]:
class Circle(object):
    def __init__(self, x, y, radius, value = 1.0, texture_frequency=1):
        self.x = x
        self.y = y
        self.radius = radius
        self.value = value
        self.texture_frequency = texture_frequency
        
    def cast(self, xs, ys, xd, yd, transparent):
        xc = self.x
        yc = self.y
        
        a = (xd - xs)**2 + (yd - ys)**2
        b = 2*((xd - xs)*(xs - xc) + (yd - ys)*(ys - yc))
        c = (xs - xc)**2 + (ys - yc)**2 - self.radius**2
        
        try:
            t0 = (-b + (b**2 - 4*a*c)**.5)/(2*a)
            t1 = (-b - (b**2 - 4*a*c)**.5)/(2*a)
        except ValueError:
            # no intersection
            return (inf, 0)
        except ZeroDivisionError:
            # source = dest
            return (inf, 0)
        
        t0m = min(t0, t1)
        t0M = max(t0, t1)
        if t0M < 0 or t0m > 1:
            return (inf, 0)
        
        if transparent:
            return min(t0, t1), self.value * abs(t1 - t0) * a**.5
        else:
            hit_t = min(t0, t1)
            xh = xs + hit_t*(xd - xs) - xc
            yh = ys + hit_t*(yd - ys) - yc
            omega = arctan2(yh, xh)
            
            return hit_t, self.value * cos(omega*self.texture_frequency)**2
    
    def draw(self, ax):
        theta = linspace(0,2*pi,128)
        ax.plot(self.x + cos(theta)*self.radius,
                self.y + sin(theta)*self.radius)

In [ ]:
v = Voxel(-1, 1, -1, 1, 50)
c = Circle(0, .5, .25)
c2 = Circle(.5, 0, .25, value=2)
c3 = Circle(0,0, .1, value=3)
c4 = Circle(-.25, -.25, .2, value=4)

v.draw(gca())
c.draw(gca())
c2.draw(gca())
c3.draw(gca())
c4.draw(gca())

axis('square')
axis([-2, 2, -2, 2])

In [ ]:
figsize(6,6)
L = v.compute_parallel([c, c2, c3, c4], transparent=True)
imshow(L, interpolation='none')
figsize(6,4)

In [ ]:
U, s, Vt = svd(L)
plot(s)

if False:
    figure()
    s[15:] = 0
    Lhat = U.dot(diag(s).dot(Vt))
    imshow(Lhat, interpolation='none')

In [ ]:
def circ_permute(L):
    L = 1*L
    N, _ = L.shape
    for j in xrange(N):
        L[:,j] = roll(L[:,j], -j)
    return L

In [ ]:
def opt_permute(L):
    L = 1*L
    N, _ = L.shape
    shifts = [0]
    for j in xrange(1,N):
        opt_err = inf
        opt_shift = 0
        for i in xrange(N):
            q = roll(L[:,j], i)
            err = ((q - L[:,j-1])**2).sum()
            if err < opt_err:
                opt_err = err
                opt_shift = i
        L[:,j] = roll(L[:,j], opt_shift)
        shifts.append(opt_shift)
    return (shifts, L)

In [ ]:
so, Lo = opt_permute(L)
Lc = circ_permute(L)
Lo = Lo
imshow(Lo, interpolation='none')

In [ ]:
U, so, Vt = svd(L)
plot(so)
plot(s)

figure()
so[11:] = 0
Loh = U.dot(diag(so).dot(Vt))
imshow(Loh)